# ตัวแทน AI ของ Azure พร้อมการสนับสนุน Model Context Protocol (MCP) - Python

สมุดบันทึกนี้แสดงวิธีการใช้ตัวแทน AI ของ Azure พร้อมเครื่องมือ Model Context Protocol (MCP) ใน Python โดยแสดงวิธีการสร้างตัวแทนอัจฉริยะที่สามารถใช้เซิร์ฟเวอร์ MCP ภายนอก (เช่น Microsoft Learn) เพื่อเพิ่มความสามารถโดยใช้การตรวจสอบสิทธิ์แบบไม่ใช้คีย์


## ติดตั้งแพ็กเกจ Python ที่จำเป็น

ก่อนอื่น เราต้องติดตั้งแพ็กเกจ Python ที่จำเป็น:
- **azure-ai-projects**: SDK หลักสำหรับ Azure AI Projects
- **azure-ai-agents**: SDK สำหรับ Azure AI Agents เพื่อสร้างและจัดการเอเจนต์
- **azure-identity**: ให้การรับรองความถูกต้องแบบไม่ใช้คีย์ด้วย DefaultAzureCredential
- **mcp**: การใช้งาน Model Context Protocol สำหรับ Python


## ข้อดีของการยืนยันตัวตนแบบไม่ใช้คีย์

สมุดบันทึกนี้แสดงตัวอย่างของ **การยืนยันตัวตนแบบไม่ใช้คีย์** ซึ่งมีข้อดีหลายประการ:
- ✅ **ไม่ต้องจัดการ API keys** - ใช้การยืนยันตัวตนตาม Azure identity
- ✅ **เพิ่มความปลอดภัย** - ไม่มีการเก็บข้อมูลลับในโค้ดหรือไฟล์การตั้งค่า
- ✅ **การหมุนเวียนข้อมูลรับรองอัตโนมัติ** - Azure จัดการวงจรชีวิตของข้อมูลรับรองให้
- ✅ **การควบคุมการเข้าถึงตามบทบาท** - ใช้ Azure RBAC เพื่อกำหนดสิทธิ์อย่างละเอียด
- ✅ **รองรับหลายสภาพแวดล้อม** - ทำงานได้อย่างราบรื่นทั้งในสภาพแวดล้อมการพัฒนาและการใช้งานจริง

`DefaultAzureCredential` จะเลือกแหล่งข้อมูลรับรองที่เหมาะสมที่สุดโดยอัตโนมัติ:
1. **Managed Identity** (เมื่อทำงานใน Azure)
2. **ข้อมูลรับรองจาก Azure CLI** (ในระหว่างการพัฒนาในเครื่อง)
3. **ข้อมูลรับรองจาก Visual Studio**
4. **ตัวแปรสภาพแวดล้อม** (ถ้ากำหนดค่าไว้)
5. **การยืนยันตัวตนผ่านเบราว์เซอร์แบบโต้ตอบ** (เป็นตัวเลือกสำรอง)


## การตั้งค่าการตรวจสอบสิทธิ์แบบไม่ใช้คีย์

**ข้อกำหนดเบื้องต้นสำหรับการตรวจสอบสิทธิ์แบบไม่ใช้คีย์:**

### สำหรับการพัฒนาบนเครื่อง:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### สำหรับสภาพแวดล้อมของ Azure:
- เปิดใช้งาน **System-assigned Managed Identity** บนทรัพยากร Azure ของคุณ
- กำหนด **RBAC roles** ที่เหมาะสมให้กับ Managed Identity:
  - `Cognitive Services OpenAI User` สำหรับการเข้าถึง Azure OpenAI
  - `AI Developer` สำหรับการเข้าถึงโครงการ Azure AI

### ตัวแปรสภาพแวดล้อม (ไม่บังคับ):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**ไม่ต้องใช้ API keys หรือ connection strings!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## นำเข้าห้องสมุดที่จำเป็น

นำเข้าโมดูล Python ที่จำเป็น:
- **os, time**: ห้องสมุดมาตรฐานของ Python สำหรับตัวแปรสภาพแวดล้อมและการหน่วงเวลา
- **AIProjectClient**: ไคลเอนต์หลักสำหรับ Azure AI Projects
- **DefaultAzureCredential**: การรับรองความถูกต้องแบบไม่ใช้กุญแจสำหรับบริการ Azure
- **MCP-related classes**: สำหรับการสร้างและจัดการเครื่องมือ MCP และการจัดการการอนุมัติ


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## กำหนดค่าการตั้งค่าเซิร์ฟเวอร์ MCP

ตั้งค่าการกำหนดค่าเซิร์ฟเวอร์ MCP โดยใช้ตัวแปรสภาพแวดล้อมพร้อมค่าเริ่มต้นสำรอง:
- **MCP_SERVER_URL**: URL ของเซิร์ฟเวอร์ MCP (ค่าเริ่มต้นคือ Microsoft Learn API)
- **MCP_SERVER_LABEL**: ป้ายกำกับเพื่อระบุเซิร์ฟเวอร์ MCP (ค่าเริ่มต้นคือ "mslearn")

วิธีนี้ช่วยให้สามารถกำหนดค่าได้อย่างยืดหยุ่นในสภาพแวดล้อมที่แตกต่างกัน


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## สร้าง Azure AI Project Client (การยืนยันตัวตนแบบไม่ใช้คีย์)

เริ่มต้นใช้งาน Azure AI Project client ด้วย **การยืนยันตัวตนแบบไม่ใช้คีย์**:
- **endpoint**: URL ของ Azure AI Foundry project endpoint
- **credential**: ใช้ `DefaultAzureCredential()` เพื่อการยืนยันตัวตนที่ปลอดภัยและไม่ต้องใช้คีย์
- **ไม่ต้องใช้ API keys**: ระบบจะค้นหาและใช้ credential ที่เหมาะสมที่สุดโดยอัตโนมัติ

**ขั้นตอนการยืนยันตัวตน:**
1. ตรวจสอบ Managed Identity (ในสภาพแวดล้อมของ Azure)
2. หากไม่มี จะใช้ Azure CLI credentials (สำหรับการพัฒนาบนเครื่องท้องถิ่น)
3. ใช้แหล่งข้อมูล credential อื่น ๆ ตามความจำเป็น

วิธีนี้ช่วยลดความยุ่งยากในการจัดการ API keys หรือ connection strings ในโค้ดของคุณ


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## สร้างคำจำกัดความเครื่องมือ MCP

สร้างเครื่องมือ MCP ที่เชื่อมต่อกับเซิร์ฟเวอร์ Microsoft Learn MCP:
- **server_label**: ตัวระบุสำหรับเซิร์ฟเวอร์ MCP
- **server_url**: จุดเชื่อมต่อ URL ของเซิร์ฟเวอร์ MCP
- **allowed_tools**: รายการตัวเลือกเพื่อจำกัดเครื่องมือที่สามารถใช้งานได้ (รายการว่างหมายถึงอนุญาตให้ใช้เครื่องมือทั้งหมด)

เครื่องมือนี้จะช่วยให้ตัวแทนสามารถเข้าถึงเอกสารและทรัพยากรของ Microsoft Learn ได้


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## สร้างเอเจนต์และดำเนินการสนทนา (เวิร์กโฟลว์แบบไม่ใช้คีย์)

ส่วนนี้แสดงขั้นตอนทั้งหมดของ **เวิร์กโฟลว์เอเจนต์แบบไม่ใช้คีย์**:

1. **สร้าง AI Agent**: ตั้งค่าเอเจนต์ด้วยโมเดล GPT-4.1 nano และเครื่องมือ MCP  
2. **สร้างเธรด**: สร้างเธรดสำหรับการสนทนา  
3. **ส่งข้อความ**: ถามเอเจนต์เกี่ยวกับความแตกต่างระหว่าง Azure OpenAI และ OpenAI  
4. **จัดการการอนุมัติเครื่องมือ**: อนุมัติการเรียกใช้เครื่องมือ MCP โดยอัตโนมัติเมื่อจำเป็น  
5. **ติดตามการดำเนินการ**: ตรวจสอบความคืบหน้าของเอเจนต์และจัดการการดำเนินการที่จำเป็น  
6. **แสดงผลลัพธ์**: แสดงรายละเอียดการสนทนาและการใช้งานเครื่องมือ  

**คุณสมบัติของเวิร์กโฟลว์แบบไม่ใช้คีย์:**
- ✅ **ไม่มีความลับที่ฮาร์ดโค้ด** - การยืนยันตัวตนทั้งหมดจัดการโดย Azure identity  
- ✅ **ปลอดภัยโดยค่าเริ่มต้น** - ใช้การควบคุมการเข้าถึงตามบทบาท  
- ✅ **การปรับใช้ง่าย** - ไม่ต้องจัดการข้อมูลรับรอง  
- ✅ **เหมาะสำหรับการตรวจสอบ** - การเข้าถึงทั้งหมดถูกติดตามผ่าน Azure identity  

เอเจนต์จะใช้เครื่องมือ MCP เพื่อเข้าถึงทรัพยากร Microsoft Learn ด้วยความปลอดภัยเต็มรูปแบบและไม่ต้องจัดการ API key


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้องมากที่สุด แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษาจากผู้เชี่ยวชาญ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่ผิดพลาดซึ่งเกิดจากการใช้การแปลนี้
